In [2]:
import pandas as pd

In [2]:
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('all-MiniLM-L6-v2')

# #Sentences are encoded by calling model.encode()
# emb1 = model.encode("Test")
# emb2 = model.encode("Test")

# cos_sim = util.cos_sim(emb1, emb2)
# print("Cosine-Similarity:", cos_sim)
# print(cos_sim.shape)

In [3]:
#READ NORMALIZED DATA FOR TARGET VERSION OF PIPELINE
df = pd.read_csv("C:\\Users\\brush\\Projekt_NLP\\normalizedResult.tsv", sep="\t")

In [4]:
#CREATE TWO LISTS THAT CONTAIN EACH TEXT OF A TWEET AND ITS TARGET
df=df.dropna()
target=(df.iloc[0:8132 , 2 ].to_list())
text=df.iloc[0:8132, 1 ].to_list()

In [5]:
#CREATE DATAFRAME THAT CONTAINS TWEET TEXT AND TARGET
df = pd.DataFrame ({'Text': text , 'Target': target })

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.Text, 
    df.Target, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2021,#-> same order of dataset random_state=2022, 2021
    stratify=df.Target
)

In [7]:
###TARGET SPLITTING OF 0.8 DATA###
targetPerson=[]
targetGroup=[]
targetPublic=[]

X_trainList = X_train.tolist()
y_trainList = y_train.tolist()

counter=0
for t in X_trainList:
    if y_trainList[counter] == 0:
        targetPerson.append(t)
    elif y_trainList[counter] == 1:
        targetGroup.append(t)
    else:
        targetPublic.append(t)
    counter+=1
###TARGET SPLITTING END###

In [8]:
###UNIQUE WORD SPLITTING OF 0.8 DATA###
personWords=[]
groupWords=[]
publicWords=[]

for tweet in targetPerson:
    personWords.append(tweet.split())
personWords = sum(personWords, [])

for tweet in targetGroup:
    groupWords.append(tweet.split())
groupWords = sum(groupWords, [])

for tweet in targetPublic:
    publicWords.append(tweet.split())
publicWords = sum(publicWords, [])

personUnique=[]
groupUnique=[]
publicUnique=[]

for word in personWords:
    if word not in personUnique:
        personUnique.append(word)
        
for word in groupWords:
    if word not in groupUnique:
        groupUnique.append(word)
        
for word in publicWords:
    if word not in publicUnique:
        publicUnique.append(word)
        
onlyPerson=[]

for word in personUnique:
    if word not in groupUnique:
        if word not in publicUnique:
            onlyPerson.append(word)
            
onlyGroup=[]

for word in groupUnique:
    if word not in personUnique:
        if word not in publicUnique:
            onlyGroup.append(word)
            
onlyPublic=[]

for word in publicUnique:
    if word not in personUnique:
        if word not in groupUnique:
            onlyPublic.append(word)
###UNIQUE WORD SPLITTING END###

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification 
import xgboost as xgb




#tfidf
#vecktorisieren

vectorizer = TfidfVectorizer() 
Xtrain = vectorizer.fit_transform(X_train) 
Xtest = vectorizer.transform(X_test) 
print(Xtrain.shape)

#vecktorisieren 



from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader


#model = SentenceTransformer('all-MiniLM-L12-v2')
#model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('all-mpnet-base-v2')
#model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
#model = SentenceTransformer('all-distilroberta-v1') multi-qa-distilbert-cos-v1
#model = SentenceTransformer('multi-qa-distilbert-cos-v1') #multi-qa-MiniLM-L6-cos-v1
#model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') #paraphrase-multilingual-mpnet-base-v2
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2') # best #paraphrase-albert-small-v2
#model = SentenceTransformer('paraphrase-albert-small-v2') #paraphrase-multilingual-MiniLM-L12-v2
#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') #paraphrase-MiniLM-L3-v2
#model = SentenceTransformer('paraphrase-MiniLM-L3-v2') #distiluse-base-multilingual-cased-v1
#odel = SentenceTransformer('distiluse-base-multilingual-cased-v1') #distiluse-base-multilingual-cased-v2
#odel = SentenceTransformer('distiluse-base-multilingual-cased-v2')

#print(X_train)
x = X_train.tolist()
Xtrain = model.encode(x) 
xt = X_test.tolist()
Xtest = model.encode(xt)

(6464, 20470)


In [13]:
#create classifier and train model
bestAcc=0
bestX=0

for x in range(21):
    y = x*10
    clf = xgb.XGBClassifier(objective="binary:logistic", random_state=2021, #booster='gblinear', 2021

    #                           n_estimators = 1000, #1000: 0.6357452071737786
                            
                            
    #                           gamma=0.9, #0.9 best with 0.6536796536796536
                            
                            
    #                           min_child_weight=19, #19 best with 0.6524427952999382 
                            
                            
    #                           learning_rate=0.2, #0.2 best with 0.6462585034013606
                            
                               #Best Acc: 0.6598639455782312 for x: 6
                               reg_alpha=y, #17 best with 0.6524427952999382
                            
                            
    #                           max_depth=11, #11 best with 0.6499690785405071


                               colsample_bytree=0.7, colsample_bylevel=0.6, colsample_bynode=1 #0.7, 0.6, 1.0 best with 0.6586270871985158
                           )

    #2. fit with X_train and y_train
    clf.fit(Xtrain, y_train)

    #3. get the predictions for X_test and store it in y_pred
    y_pred = clf.predict(Xtest)

#     #4. classfication report BEFORE ANALYZER
#     classificationBefore = classification_report(y_test, y_pred)

    #5. find accuracy scores
    accuracyBefore = accuracy_score(y_test, y_pred)
    print("Current Acc: " + str(accuracyBefore) + " for x: " + str(y))
    if accuracyBefore > bestAcc:
        bestAcc = accuracyBefore
        bestX = y
print("Best Acc: " + str(bestAcc) + " for x: " + str(bestX))

Current Acc: 0.6586270871985158 for x: 0
Current Acc: 0.645021645021645 for x: 10
Current Acc: 0.6444032158317873 for x: 20
Current Acc: 0.6388373531230674 for x: 30
Current Acc: 0.6283240568954854 for x: 40
Current Acc: 0.6277056277056277 for x: 50
Current Acc: 0.6178107606679035 for x: 60
Current Acc: 0.616573902288188 for x: 70
Current Acc: 0.6110080395794681 for x: 80
Current Acc: 0.6091527520098948 for x: 90


KeyboardInterrupt: 

In [33]:
#6. count prediction targets
personIst=0
groupIst=0
publicIst=0
for p in y_pred:
    if p == 0:
        personIst+=1
    if p == 1:
        groupIst+=1
    if p == 2:
        publicIst+=1

In [1]:
print(y_pred[0:100])
# y_pred2=y_pred
# countList=[]
# personCount=0
# groupCount=0
# publicCount=0
# counter=0
# single_pronouns=["du", "dich", "deinen"]
# single_adress=["herr", "herrn", "frau", "junge", "nazi"]
# single_politicians=["merkel", "söder", "spahn", "soeder", "laschet", "baerbock", "bundeskanzlerin", "bundeskanzler", "kanzler", "kanzlerin"]
# group_pronouns=["euch", "eure", "euer", "deren", "ihr"]
# group_politics=["spd", "cdu", "csu", "afd", "grünen", "grüne", "union", "linken", "nazis", "rechten", "partei", "land", "terroristen", "demokratie"]
# group_countries=["deutschland", "deutsche", "bürger"]
# public_pronouns=["alle", "wir", "menschen", "volk", "bevölkerung", "welt", "bewohner"]
# testList=public_pronouns
# Xtest = Xtest.array2string()

NameError: name 'y_pred' is not defined

In [13]:
# #7. run analyzer for target correction
# #7.1 filter out person tweets out of group tweets and correct prediction
# print(type(Xtest))
# print(type(Xtest[0]))
# for s in y_pred2:
#     if y_pred2[counter] == 1:
#         doc=Xtest[counter].split()
#         for token in doc:
#             if token in onlyPerson:
#                 personCount+=1
#             if token in single_pronouns or token in single_adress or token in single_politicians:
#                 personCount+=1
#         #7       (0.6239950525664811)
#         #6       (0.6246134817563389)
#         #5       (0.6246134817563389) <-- BEST
#         #4       (0.6246134817563389)
#         #3       (0.6239950525664811)
#         if personCount >= 5:
#             y_pred2[counter]=0
#         personCount=0
#     counter+=1
# counter=0

# #7.1.1. classfication report AFTER ANALYZER (Step 1/3)
# classificationAfter = classification_report(y_test, y_pred2)

# #7.1.2. find accuracy scores
# accuracyAfter = accuracy_score(y_test, y_pred2)

# #7.1.3. count prediction targets
# personIst2=0
# groupIst2=0
# publicIst2=0
# for p in y_pred2:
#     if p == 0:
#         personIst2+=1
#     if p == 1:
#         groupIst2+=1
#     if p == 2:
#         publicIst2+=1

# #7.2. filter out group tweets out of person tweets and correct prediction
# for s in y_pred2:
#     if y_pred2[counter] == 0:
#         doc=Xtest[counter].split()
#         for token in doc:
#             if token in onlyGroup:
#                 groupCount+=1
#             if token in group_pronouns or token in group_politics or token in group_countries:
#                 groupCount+=1
#         #6       (0.6246134817563389)
#         #5       (0.6252319109461967) <-- BEST
#         #4       (0.6252319109461967)
#         #3       (0.6246134817563389)
#         #2       (0.6171923314780458)
#         if groupCount >= 5:
#             y_pred2[counter]=1
#         groupCount=0
#     counter+=1
# counter=0

# #7.2.1. classfication report AFTER ANALYZER (Step 2/3)
# classificationAfter2 = classification_report(y_test, y_pred2)

# #7.2.2. find accuracy scores
# accuracyAfter2 = accuracy_score(y_test, y_pred2)

# #7.2.3. count prediction targets
# personIst3=0
# groupIst3=0
# publicIst3=0
# for p in y_pred2:
#     if p == 0:
#         personIst3+=1
#     if p == 1:
#         groupIst3+=1
#     if p == 2:
#         publicIst3+=1

# #7.3. filter out public tweets out of person and group tweets and correct prediction
# for s in y_pred2:
#     if y_pred2[counter] == 0 or y_pred2[counter] == 1:
#         doc=Xtest[counter].split()
#         for token in doc:
#             if token in onlyPublic:
#                 publicCount+=1
#             if token in public_pronouns:
#                 publicCount+=1
#         #6       (0.6252319109461967)
#         #5       (0.6258503401360545) <-- BEST
#         #4       (0.6258503401360545)
#         #3       (0.6190476190476191)
#         if publicCount >= 5:
#             y_pred2[counter]=2
#         publicCount=0
#     counter+=1
# counter=0

# #7.3.1. classfication report AFTER ANALYZER (Step 3/3)
# classificationAfter3 = classification_report(y_test, y_pred2)

# #7.3.2. find accuracy scores
# accuracyAfter3 = accuracy_score(y_test, y_pred2)

# #7.3.3. count prediction targets
# personIst4=0
# groupIst4=0
# publicIst4=0
# for p in y_pred2:
#     if p == 0:
#         personIst4+=1
#     if p == 1:
#         groupIst4+=1
#     if p == 2:
#         publicIst4+=1

In [34]:
###PRINT ALL CLASSIFICATION REPORTS, ACCURACY SCORES AND TARGET COUNTS###
print("BEFORE ANALZYER:")
print(classificationBefore)
print("The accuracy of prediction is: ", accuracyBefore)
print("IST-WERT BEFORE ANALYZER:")
print("Person: " + str(personIst))
print("Group: " + str(groupIst))
print("Public: " + str(publicIst))
print("###############################################")
# print("AFTER ANALYZER STEP 1:")
# print(classificationAfter)
# print("The accuracy of prediction is: ", accuracyAfter)
# print("IST-WERT AFTER STEP 1:")
# print("Person: " + str(personIst2))
# print("Group: " + str(groupIst2))
# print("Public: " + str(publicIst2))
# print("AFTER ANALYZER STEP 2:")
# print(classificationAfter2)
# print("The accuracy of prediction is: ", accuracyAfter2)
# print("IST-WERT AFTER STEP 2:")
# print("Person: " + str(personIst3))
# print("Group: " + str(groupIst3))
# print("Public: " + str(publicIst3))
# print("AFTER ANALYZER")
# print(classificationAfter3)
# print("The accuracy of prediction is: ", accuracyAfter3)
# print("IST-WERT AFTER")
# print("Person: " + str(personIst4))
# print("Group: " + str(groupIst4))
# print("Public: " + str(publicIst4))

BEFORE ANALZYER:
              precision    recall  f1-score   support

           0       0.69      0.70      0.69       571
           1       0.65      0.75      0.70       642
           2       0.62      0.45      0.52       404

    accuracy                           0.66      1617
   macro avg       0.65      0.63      0.64      1617
weighted avg       0.66      0.66      0.65      1617

The accuracy of prediction is:  0.6586270871985158
IST-WERT BEFORE ANALYZER:
Person: 582
Group: 742
Public: 293
###############################################
